# Importing Libraries

In [ ]:
import os
import torch
import warnings

import pandas as pd

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

warnings.filterwarnings('ignore')

In [ ]:
OUTPUT_DIR = "results"
LOG_DIR = "logs"

TRAIN_FILE = "dataset/train.txt"  # Path to your training data file
VAL_FILE = "dataset/val.txt"  # Path to your validation data file
TEST_FILE = "dataset/test.txt"  # Path to your training data file

# Loading Pre-Trained Model

In [ ]:
# Load model directly
from transformers import AutoProcessor, AutoModelForImageTextToText

processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")
model = AutoModelForImageTextToText.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

In [ ]:
# Load FLAN-T5
MODEL_NAME = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

model.gradient_checkpointing_enable()

model.config.resid_pdrop = 0.1  # Add dropout
model.config.embd_pdrop = 0.1
model.config.attn_pdrop = 0.1

model.resize_token_embeddings(len(tokenizer))

In [ ]:
torch.mps.empty_cache()

In [ ]:
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
model.to(device)

In [ ]:
def count_tokens(file_path, tokenizer):
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
    tokens = tokenizer.tokenize(text)
    return len(tokens)

train_token_count = count_tokens("dataset/train.txt", tokenizer)
eval_token_count = count_tokens("dataset/val.txt", tokenizer)

print(train_token_count)
print(eval_token_count)

# Training the model


## Setting up training arguments

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,  # Reduce batch size if necessary
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
    logging_dir="logs",

    fp16=False,  # Ensure mixed precision is disabled
    bf16=False,  # Disable bf16 as well
    half_precision_backend="cpu"  # Force precision handling on CPU (not MPS)
)

## Loading the dataset

In [ ]:
def load_chatbot_data(file_path):
    """Load and preprocess chatbot data from the given text file."""
    conversations = []
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
        user_input, bot_response = None, None
        for line in lines:
            if line.startswith("User:"):
                user_input = line.replace("User:", "").strip()
            elif line.startswith("Bot:"):
                bot_response = line.replace("Bot:", "").strip()
                if user_input and bot_response:
                    conversations.append({"input": user_input, "output": bot_response})
                    user_input, bot_response = None, None
    return pd.DataFrame(conversations)

# Prepare datasets
df_train = load_chatbot_data(TRAIN_FILE)
df_val = load_chatbot_data(VAL_FILE)

dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)

In [ ]:
print(f"Length of train_dataset: {len(dataset_train)}")
print(f"Length of eval_dataset: {len(dataset_val)}")

In [ ]:
def tokenize_function(examples):
    model_inputs = tokenizer(examples["input"], max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(examples["output"], max_length=512, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    
    model_inputs = {key: torch.tensor(val).to(device) for key, val in model_inputs.items()}  

    return model_inputs

# Apply tokenization
tokenized_train = dataset_train.map(tokenize_function, batched=True)
tokenized_val = dataset_val.map(tokenize_function, batched=True)

## Initialising the trainer

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
torch.mps.empty_cache()

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
# Train the model
trainer.train()

## Saving the model

In [ ]:
model.save_pretrained("results/flan_t5_chatbot")
tokenizer.save_pretrained("results/flan_t5_chatbot")

# Evaluation

In [ ]:
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
model.to(device)  # Move model to MPS

def chat_with_model(prompt, max_length=100):
    """Generate a response from the trained chatbot model."""
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=1024)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    output = model.generate(**inputs, max_length=max_length, num_return_sequences=1)
    response = tokenizer.decode(output[0].cpu(), skip_special_tokens=True)
    
    return response

if __name__ == "__main__":
    print("Chatbot is ready! Type 'exit' to stop.")
    while True:
        user_input = input("User: ")
        if user_input.lower() == "exit":
            break
        bot_response = chat_with_model(user_input)
        print(f"User: {user_input}")
        print(f"Bot: {bot_response}")

In [ ]:
def chatbot_response(prompt, max_length=100):
    # Tokenize input prompt
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Generate response
    outputs = model.generate(**inputs, max_length=max_length)

    # Decode the generated text
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


In [ ]:
user_input = "how do i cure depression?"
response = chatbot_response(user_input)
print("Chatbot:", response)

# Model for translation

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small")